In [5]:
import rasterio
import numpy as np
import os
from rasterio.mask import mask
from shapely.geometry import box

# Catchment IDs
catchment_no = np.arange(1, 23)
exclude = [11, 17, 18, 19]
catchs = np.setdiff1d(catchment_no, exclude)

# Resolutions and vegetation types
resos = [20, 40, 80, 160]
trees = ['spruce', 'pine', 'decid']

# Base folders
ch_base = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed'
lai_base = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed'
cf_base = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed'
soil_base = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD'

for reso in resos:
    # Define input file paths
    ch_path = os.path.join(ch_base, f'canopy_height_{reso}.asc')
    cf_path = os.path.join(cf_base, f'canopy_fraction_{reso}.asc')
    soil_path = os.path.join(soil_base, f'krycklan_QD_J1_J2_final_{reso}.asc')
    lake_path = os.path.join(soil_base, f'lake_mask_{reso}.asc')
    
    lai_paths = [os.path.join(lai_base, f'LAI_{tree}_{reso}.asc') for tree in trees]
    files = [ch_path, cf_path, soil_path, lake_path] + lai_paths

    for catch in catchs:
        ref_fp = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}/cmask.tif'
        
        # Open reference cmask
        with rasterio.open(ref_fp) as ref_src:
            cmask = ref_src.read(1).astype("float32")
            bbox = box(*ref_src.bounds)
            shapes = [bbox]
            ref_meta = ref_src.meta.copy()

        for file in files:
            # Clean filename (strip resolution suffix)
            filename = os.path.basename(file)
            for r in resos:
                filename = filename.replace(f"_{r}", "")
            out_fp = os.path.join(os.path.dirname(ref_fp), filename)

            with rasterio.open(file) as src:
                try:
                    clipped_data, clipped_transform = mask(src, shapes, crop=True)
                except ValueError:
                    print(f"⚠️ Skipping empty intersection: {file} for catchment C{catch} ({reso}m)")
                    continue

                # Use first band (2D)
                clipped_band = clipped_data[0]

                # Crop cmask to same shape as clipped raster
                cmask_cropped = cmask[:clipped_band.shape[0], :clipped_band.shape[1]]

                # Apply mask: assign -9999 where cmask
                masked_data = np.where(np.isnan(cmask_cropped), -9999, clipped_band).astype("float32")
                masked_data = np.where(cmask_cropped == -9999, -9999, clipped_band).astype("float32")

                # Update metadata for single-band raster
                clipped_meta = src.meta.copy()
                clipped_meta.update({
                    "height": masked_data.shape[0],
                    "width": masked_data.shape[1],
                    "transform": clipped_transform,
                    "nodata": -9999,
                    "dtype": "float32",
                    "count": 1
                })

                # Write 2D array directly
                with rasterio.open(out_fp, 'w', **clipped_meta) as dst:
                    dst.write(masked_data, 1)

            print(f"✅ Clipped & masked: {filename}, C{catch} & {reso}m")


✅ Clipped & masked: canopy_height.asc, C1 & 20m
✅ Clipped & masked: canopy_fraction.asc, C1 & 20m
✅ Clipped & masked: krycklan_QD_J1_J2_final.asc, C1 & 20m
✅ Clipped & masked: lake_mask.asc, C1 & 20m
✅ Clipped & masked: LAI_spruce.asc, C1 & 20m
✅ Clipped & masked: LAI_pine.asc, C1 & 20m
✅ Clipped & masked: LAI_decid.asc, C1 & 20m
✅ Clipped & masked: canopy_height.asc, C2 & 20m
✅ Clipped & masked: canopy_fraction.asc, C2 & 20m
✅ Clipped & masked: krycklan_QD_J1_J2_final.asc, C2 & 20m
✅ Clipped & masked: lake_mask.asc, C2 & 20m
✅ Clipped & masked: LAI_spruce.asc, C2 & 20m
✅ Clipped & masked: LAI_pine.asc, C2 & 20m
✅ Clipped & masked: LAI_decid.asc, C2 & 20m
✅ Clipped & masked: canopy_height.asc, C3 & 20m
✅ Clipped & masked: canopy_fraction.asc, C3 & 20m
✅ Clipped & masked: krycklan_QD_J1_J2_final.asc, C3 & 20m
✅ Clipped & masked: lake_mask.asc, C3 & 20m
✅ Clipped & masked: LAI_spruce.asc, C3 & 20m
✅ Clipped & masked: LAI_pine.asc, C3 & 20m
✅ Clipped & masked: LAI_decid.asc, C3 & 20m
✅ Cl

In [ ]:
import os
import rasterio
from rasterio.merge import merge
import numpy as np

# Root directory
root_dir = '/Users/jpnousu/Krycklan_GIS_data/20m'
all_dir = os.path.join(root_dir, "ALL")

# ✅ Create ALL folder if not exists
os.makedirs(all_dir, exist_ok=True)

# Get subcatchment directories (C1..CX)
subcatchments = [d for d in os.listdir(root_dir) 
                 if os.path.isdir(os.path.join(root_dir, d)) and d.startswith("C")]

if not subcatchments:
    raise RuntimeError("No subcatchment folders found!")

# Use first catchment (C1) to determine which .asc files to mosaic
sample_dir = os.path.join(root_dir, subcatchments[0])
asc_files = [f for f in os.listdir(sample_dir) if f.endswith(".asc")]

print(f"Found {len(asc_files)} ASC file types to mosaic: {asc_files}")

for asc_file in asc_files:
    print(f"\n▶ Mosaicing {asc_file} ...")

    # Collect paths for this file across all subcatchments
    asc_paths = []
    for sub in subcatchments:
        candidate = os.path.join(root_dir, sub, asc_file)
        if os.path.exists(candidate):
            asc_paths.append(candidate)

    if not asc_paths:
        print(f"  ⚠️ Skipping {asc_file} (not found in any subcatchment)")
        continue

    # Open all rasters
    src_files_to_mosaic = [rasterio.open(p) for p in asc_paths]

    # Merge rasters
    mosaic, out_transform = merge(src_files_to_mosaic)

    # Copy metadata
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "AAIGrid",      # keep ASCII grid format
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform,
        "nodata": -9999,
        "dtype": "float32",
        "count": 1
    })

    # Replace any nodata with -9999
    data = np.where(mosaic == out_meta.get("nodata", None), -9999, mosaic).astype("float32")

    # Output path in ALL
    out_path = os.path.join(all_dir, asc_file)

    # Save as ASC
    with rasterio.open(out_path, "w", **out_meta) as dest:
        dest.write(data)

    print(f"  ✅ Saved mosaic to {out_path}")

In [6]:
import os
import rasterio
from rasterio.merge import merge
import numpy as np

# Root directory (base path without resolution)
root_base = '/Users/jpnousu/Krycklan_GIS_data'
resos = [20, 40, 80, 160]  # resolutions to loop over

for reso in resos:
    print(f"\n=== Processing {reso}m resolution ===")

    root_dir = os.path.join(root_base, f"{reso}m")
    all_dir = os.path.join(root_dir, "ALL")
    os.makedirs(all_dir, exist_ok=True)

    # Get subcatchment directories (C1..CX)
    subcatchments = [d for d in os.listdir(root_dir) 
                     if os.path.isdir(os.path.join(root_dir, d)) and d.startswith("C")]

    if not subcatchments:
        raise RuntimeError(f"No subcatchment folders found in {root_dir}!")

    # Use first catchment to determine which .asc files to mosaic
    sample_dir = os.path.join(root_dir, subcatchments[0])
    asc_files = [f for f in os.listdir(sample_dir) if f.endswith(".asc")]

    print(f"Found {len(asc_files)} ASC file types to mosaic: {asc_files}")

    for asc_file in asc_files:
        print(f"\n▶ Mosaicing {asc_file} ...")

        # Collect paths for this file across all subcatchments
        asc_paths = []
        for sub in subcatchments:
            candidate = os.path.join(root_dir, sub, asc_file)
            if os.path.exists(candidate):
                asc_paths.append(candidate)

        if not asc_paths:
            print(f"  ⚠️ Skipping {asc_file} (not found in any subcatchment)")
            continue

        # Open all rasters
        src_files_to_mosaic = [rasterio.open(p) for p in asc_paths]

        # Merge rasters
        mosaic, out_transform = merge(src_files_to_mosaic)

        # Copy metadata
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "driver": "AAIGrid",      # keep ASCII grid format
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform,
            "nodata": -9999,
            "dtype": "float32",
            "count": 1
        })

        # Replace any nodata with -9999
        data = np.where(mosaic == out_meta.get("nodata", None), -9999, mosaic).astype("float32")

        # Output path in ALL
        out_path = os.path.join(all_dir, asc_file)

        # Save as ASC
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(data)

        print(f"  ✅ Saved mosaic to {out_path}")


=== Processing 20m resolution ===
Found 14 ASC file types to mosaic: ['ManuallyDigitizedDitchesKrycklan.asc', 'LAI_pine.asc', 'twi_dinf.asc', 'channels.asc', 'canopy_fraction.asc', 'cmask.asc', '5haStreams.asc', 'twi_d8.asc', 'LAI_decid.asc', 'lake_mask.asc', 'dem_clip_culv_fill_streams.asc', 'krycklan_QD_J1_J2_final.asc', 'LAI_spruce.asc', 'canopy_height.asc']

▶ Mosaicing ManuallyDigitizedDitchesKrycklan.asc ...
  ✅ Saved mosaic to /Users/jpnousu/Krycklan_GIS_data/20m/ALL/ManuallyDigitizedDitchesKrycklan.asc

▶ Mosaicing LAI_pine.asc ...
  ✅ Saved mosaic to /Users/jpnousu/Krycklan_GIS_data/20m/ALL/LAI_pine.asc

▶ Mosaicing twi_dinf.asc ...
  ✅ Saved mosaic to /Users/jpnousu/Krycklan_GIS_data/20m/ALL/twi_dinf.asc

▶ Mosaicing channels.asc ...
  ✅ Saved mosaic to /Users/jpnousu/Krycklan_GIS_data/20m/ALL/channels.asc

▶ Mosaicing canopy_fraction.asc ...
  ✅ Saved mosaic to /Users/jpnousu/Krycklan_GIS_data/20m/ALL/canopy_fraction.asc

▶ Mosaicing cmask.asc ...
  ✅ Saved mosaic to /Users

In [11]:
import os
import rasterio
import numpy as np

# Config
resos = [20, 40, 80, 160]
species = ['spruce', 'decid', 'pine']
nodata_out = -9999.0
propagate_any_nodata = False  # If True -> output nodata where ANY species is nodata

for reso in resos:
    print(f"\n▶ Processing LAI total for {reso}m ...")

    # Build paths
    lai_paths = [
        f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/LAI_{specie}.asc'
        for specie in species
    ]

    data_list = []
    valid_masks = []
    meta = None

    # Read species rasters
    for fp in lai_paths:
        if not os.path.exists(fp):
            raise FileNotFoundError(f"Missing LAI file: {fp}")

        with rasterio.open(fp) as src:
            arr = src.read(1).astype("float32")
            src_nodata = src.nodata if src.nodata is not None else nodata_out

            # Build valid-data mask
            valid = arr != src_nodata

            # For summing, treat nodata as 0.0
            arr_filled = np.where(valid, arr, 0.0).astype("float32")

            data_list.append(arr_filled)
            valid_masks.append(valid)

            # Save metadata from first raster
            if meta is None:
                meta = src.meta.copy()

    if meta is None:
        print(f"  ⚠️ No metadata found for {reso}m — skipping")
        continue

    # Sanity check: ensure all arrays have same shape
    shapes = {d.shape for d in data_list}
    if len(shapes) != 1:
        raise RuntimeError(f"Input LAI rasters for {reso}m have different shapes: {shapes}")

    # Sum species (nodata treated as 0 for sum)
    lai_total = np.sum(data_list, axis=0).astype("float32")

    # Determine where output should be nodata
    if propagate_any_nodata:
        # nodata where any species is nodata (strict propagation)
        any_nodata_mask = np.logical_not(np.logical_and.reduce(valid_masks))
        out_mask_nodata = any_nodata_mask
    else:
        # nodata only where ALL species are nodata (i.e. no data anywhere)
        any_valid = np.logical_or.reduce(valid_masks)
        out_mask_nodata = np.logical_not(any_valid)

    # Assign nodata_out where masked
    lai_total = np.where(out_mask_nodata, nodata_out, lai_total).astype("float32")

    # Update metadata for single-band output ASCII
    meta.update({
        "dtype": "float32",
        "nodata": nodata_out,
        "count": 1,
        "driver": "AAIGrid"
    })

    out_fp = f'/Users/jpnousu/Krycklan_GIS_data/{reso}m/ALL/LAI_tot.asc'
    with rasterio.open(out_fp, "w", **meta) as dst:
        dst.write(lai_total, 1)

    print(f"  ✅ Saved LAI total: {out_fp}")



▶ Processing LAI total for 20m ...
  ✅ Saved LAI total: /Users/jpnousu/Krycklan_GIS_data/20m/ALL/LAI_tot.asc

▶ Processing LAI total for 40m ...
  ✅ Saved LAI total: /Users/jpnousu/Krycklan_GIS_data/40m/ALL/LAI_tot.asc

▶ Processing LAI total for 80m ...
  ✅ Saved LAI total: /Users/jpnousu/Krycklan_GIS_data/80m/ALL/LAI_tot.asc

▶ Processing LAI total for 160m ...
  ✅ Saved LAI total: /Users/jpnousu/Krycklan_GIS_data/160m/ALL/LAI_tot.asc
